In [ ]:
import websocket
import websockets
from websocket import create_connection
import json
from scipy.optimize import minimize
from scipy.optimize import basinhopping
from random import randrange
import asyncio

### Initialize variables

In [ ]:
current_state_msg = None
optimization_result_msg = None
sol = None
current_pit_volume = 0
pit_max_capacity = 250000
lower_bound_percentage_on_flow_rates = 0.80
count = 0

### Handle message from the server and take action according to the msg type

In [ ]:
def handle_message(message):
    global current_pit_volume
    global current_state_msg
    global optimization_result_msg
    global count
    try:
        message = json.loads(message)
        # print('handle_message')
        if message['type'] == 'CURRENT_STATE':
            # print('CURRENT_STATE msg received')
            current_state_msg = message
            print('Flow rate in: ' + str(current_state_msg['flowRateIn']) + ' + ' + str(current_pit_volume))
            current_state_msg['flowRateIn'] += current_pit_volume
            calc_optimum_values()
        elif message['type'] == 'OPTIMATION_RESULT':
            print('OPTIMATION_RESULT msg received')
            print(message)
            optimization_result_msg = message
            current_pit_volume = optimization_result_msg['currentPitVolume']
            
            f = open(str(count) + ".txt", "w")
            optimization_result_msg['maximized_output'] = sol.fun * -1

            for key, value in optimization_result_msg.items():
                if type(value) == int or type(value) == float:
                    optimization_result_msg[key] = round(value, 2)


            f.write(json.dumps(optimization_result_msg))
            count += 1
            
    except Exception as e:
        print('Exception: ' + str(e) + ' ' + str(message))
        



### Objective func and constraints for basinhopping algorithm

In [ ]:
# Maximizing this objective function
def get_dollars(flow_rates):

    final_flow_rates = []

    for ind in range(len(current_state_msg['operations'])):
        
        i = 0
        while True:
            curr_value = current_state_msg['operations'][ind]['revenueStructure'][i]
            i += 1
            # print(str(curr_value) + ' ' + str(flow_rates[ind]) + ' ' + str(i))
            if flow_rates[ind] <= curr_value['flowPerDay'] or i >= len(current_state_msg['operations'][ind]['revenueStructure']):
                break
        
        if i == 1:
            final_flow_rates.append(curr_value['dollarsPerDay'])
        else:
            # Linear interpolation between two points
            # y2, y1, x2, x1 = curr_value['dollarsPerDay'], current_state_msg['operations'][ind]['revenueStructure'][i - 2]['dollarsPerDay'], curr_value['flowPerDay'], current_state_msg['operations'][ind]['revenueStructure'][i - 2]['flowPerDay']
            y2 = curr_value['dollarsPerDay']
            y1 = current_state_msg['operations'][ind]['revenueStructure'][i - 2]['dollarsPerDay']
            x2 = curr_value['flowPerDay']
            x1 = current_state_msg['operations'][ind]['revenueStructure'][i - 2]['flowPerDay']
            m = (y2 - y1) / (x2 - x1)
            c = y1 - m * x1
            final_flow_rates.append(m * flow_rates[ind] + c)
    
    # Return with -ve sign to convert maximization to minimization problem.
    return -1 * sum(final_flow_rates)

# Constraint of sum of all flow Rate To Operations to be less than equal to flow rate in
def constraint_func(flow_rates):
    return current_state_msg['flowRateIn'] - sum(flow_rates)

# Constraint of sum of all flow Rate To Operations to be greater than equal to lower_bound_percentage_on_flow_rates fraction of flow rate in
def constraint_func2(flow_rates):
    return sum(flow_rates) - lower_bound_percentage_on_flow_rates * current_state_msg['flowRateIn']


### calc_optimum_values optimizes flow rates to maximize revenue

In [ ]:
def calc_optimum_values():
    global sol
    #print('calc_optimum_values')

    bnds = tuple((0, 200000) for i in range(len(current_state_msg['operations'])))
    x0 = [randrange(20000) for i in range(len(current_state_msg['operations']))]
    con1 = {'type': 'ineq', 'fun': constraint_func}
    con2 = {'type': 'ineq', 'fun': constraint_func2}
    # cons = [con1]
    cons = [con1, con2]
    #print("before basinhopping")

    # Ignore minimize
    # sol = minimize(get_dollars, x0, method='SLSQP', bounds = bnds, constraints = cons, options={'maxiter': 200})

    # Executing basinhopping algorithm
    sol = basinhopping(get_dollars, x0, niter=100, minimizer_kwargs = {"bounds": bnds, "constraints":cons, "method":'SLSQP'})
    #print("after basinhopping")

    print('Maximized revenue: $' + str(sol.fun * -1))
    print('Water not used: ' + str(current_state_msg['flowRateIn'] - sum(sol.x)) + ' bbls')


    # Flow rate to operations data to be sent to server
    data_to_send = []

    for ind in range(len(current_state_msg['operations'])):
        data_to_send.append({'operationId': current_state_msg['operations'][ind]['id'], 'flowRate': sol.x[ind]})

    print('Send to server: ' + json.dumps(data_to_send))
    ws.send(json.dumps(data_to_send))
    

### Listeners for websocket connection to server

In [ ]:
def on_message(ws, message):
    #print("Received message")
    #print(message)
    handle_message(message)

def on_error(ws, error):
    print(error)

def on_close(ws, close_status_code, close_msg):
    print("### closed ###")

def on_open(ws):
    print("connection opened")
    ws.send(json.dumps({'setPitCapacity': pit_max_capacity}))

### Initialize and run the websocket

In [ ]:
websocket.enableTrace(False)
ws = websocket.WebSocketApp("wss://2021-utd-hackathon.azurewebsites.net",
                            on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

ws.run_forever()